# Functional example for LUME-Impact

In [ ]:
from impact import Impact, run_impact_with_distgen, evaluate_impact_with_distgen
import h5py
import os

# Nicer plotting
import matplotlib

%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams["figure.figsize"] = (8, 4)

In [ ]:
gfile = "templates/lcls_injector/distgen.yaml"
ifile = "templates/lcls_injector/ImpactT.yaml"

# Functional run_impact_with_distgen

This is a functional way to apply some settings to and run distgen and impact together.

Any key with a prefix `distgen:` will send its suffix to distgen's Generator.

Otherwise, any key that can be set with Impact's attr syntax can be used.



In [ ]:
# Make some settings
SETTINGS0 = {
    "distgen:n_particle": 100,
    "total_charge": 0,  # effectively turns spacecharge off
    "stop": 0.02,
    "GUN_phase:autophase_deg": 0,
}


# This returns an Impact object that has run
I = run_impact_with_distgen(
    settings=SETTINGS0,
    distgen_input_file=gfile,
    impact_config=ifile,
)

I.plot("mean_kinetic_energy")
I

In [ ]:
h5file = I.archive()

In [ ]:
with h5py.File(h5file, "r") as h5:
    print(list(h5))

In [ ]:
I.from_archive(h5file)
I.output.keys()

In [ ]:
I["total_charge"] = 1
I.write_input()
!cat {I.path}/ImpactT.in

# Functional evaluate_impact_with_distgen

Similar to above, but reuruns a dict of common outputs, and an optinal archive file.

This is useful in optimizations and creating datasets.

In [ ]:
# This returns an Impact object that has run
O1 = evaluate_impact_with_distgen(
    SETTINGS0, distgen_input_file=gfile, impact_config=ifile, archive_path="."
)

# This is the default output
O1

In [ ]:
I2 = Impact.from_archive(O1["archive"])

In [ ]:
I2.fingerprint() == I.fingerprint()

A custom merit function can be provided to give different output. Note that the fingerprint is always returned.

In [ ]:
# Custom merit function
def my_merit(impact_object):
    """
    Custom merit function.

    Returns the maximum sigma_x seen

    """
    d = {"max_sigma_x": I.stat("sigma_x").max()}

    return d


evaluate_impact_with_distgen(
    SETTINGS0, distgen_input_file=gfile, impact_config=ifile, merit_f=my_merit
)

In [ ]:
# Cleanup
os.remove(O1["archive"])
os.remove(h5file)